In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

sns.set(style="whitegrid", color_codes=True, font_scale=1.3)

%matplotlib inline

df = pd.read_csv(r"C:\Users\Rahul Aggarwal\Desktop\FIFA_20.csv")
# df.head()

df.describe().T

for i in range(25) : # len(df)
  print(df.loc[i, "short_name"], df.loc[i, "club"])

<h1> Data Cleaning </h1>

df.drop(df.iloc[:, 78:104], inplace = True, axis = 1) 

df = df.drop(['sofifa_id', 'player_url', 'long_name', 'dob', 'real_face',
         'player_positions', 'work_rate', 'player_tags', 'loaned_from',
         'joined', 'contract_valid_until', 'nation_position',
         'nation_jersey_number', 'player_traits'], axis = 1)

sub_count = 0

for i in range(len(df)):
  if (df.loc[i, 'team_position'] == 'SUB'):
        sub_count += 1
        # print(df.loc[i, "short_name"], df.loc[i, "club"], df.loc[i, 'team_position'])
        

print('Total subs :', sub_count)

# Dropping subs to analyze only starting 11 team players
starting_eleven_df = df[df['team_position'] != 'SUB']

print(df.shape)
print(starting_eleven_df.shape)

starting_eleven_df.head()

starting_eleven_df = starting_eleven_df.reset_index(drop = True)

import sklearn.cluster as cluster
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt


cols = ['age', 'potential']

df_test = df[cols]


x = df_test['age']
y = df_test['potential']
# Generate indicators for the data points
obj_names = []
for i in range(len(df)):
    obj = "Object " + str(i)
    obj_names.append(obj)

data = pd.DataFrame({
    'Object': obj_names,
    'X_value': x,
    'Y_value': y
})

print(df.head())

plt.scatter(x, y, s = 5)
plt.xlabel('age')
plt.ylabel('potential')
plt.show()


# Initialize the centroids
# (age, potential)
c1 = (20, 55)
c2 = (30, 85)
c3 = (35, 60)

# Function to calculate distances of all points from a particular centroid
def calculate_distance(centroid, X, Y):
    distances = []
    
    # Unpack the x and y coordinates of the centroid
    c_x, c_y = centroid
    
    # Iterate over the data points and calculate the distance using the given formula
    for x, y in list(zip(X, Y)):
        root_diff_x = (x - c_x) ** 2
        root_diff_y = (y - c_y) ** 2
        distance = np.sqrt(root_diff_x + root_diff_y)
        distances.append(distance)
    
    return distances

# Calculate the distance and assign them to the DataFrame accordingly
data['C1_Distance'] = calculate_distance(c1, data.X_value, data.Y_value)
data['C2_Distance'] = calculate_distance(c2, data.X_value, data.Y_value)
data['C3_Distance'] = calculate_distance(c3, data.X_value, data.Y_value)

# Preview the data
data.head()

# Get the minimum distance centroids
data['Cluster'] = data[['C1_Distance', 'C2_Distance', 'C3_Distance']].apply(np.argmin, axis =1)

# Map the centroids accordingly and rename them
data['Cluster'] = data['Cluster'].map({'C1_Distance': 'C1', 'C2_Distance': 'C2', 'C3_Distance': 'C3'})

# Get a preview of the data
data.head(5)

# Get the number of data points in the clusters
from collections import Counter

label_counts = Counter(data['Cluster'].values)
label_counts.most_common()

# Calculate the coordinates of the new centroid from cluster 1
x_new_centroid1 = data[data['Cluster']=='C1']['X_value'].mean()
y_new_centroid1 = data[data['Cluster']=='C1']['Y_value'].mean()

# Calculate the coordinates of the new centroid from cluster 2
x_new_centroid2 = data[data['Cluster']=='C2']['X_value'].mean()
y_new_centroid2 = data[data['Cluster']=='C2']['Y_value'].mean()

# Calculate the coordinates of the new centroid from cluster 3
x_new_centroid3 = data[data['Cluster']=='C3']['X_value'].mean()
y_new_centroid3 = data[data['Cluster']=='C3']['Y_value'].mean()

# Print the coordinates of the new centroids
print('Centroid 1 ({}, {})'.format(x_new_centroid1, y_new_centroid1))
print('Centroid 2 ({}, {})'.format(x_new_centroid2, y_new_centroid2))
print('Centroid 3 ({}, {})'.format(x_new_centroid3, y_new_centroid3))

# Using scikit-learn to perform K-Means clustering
from sklearn.cluster import KMeans

# Specify the number of clusters (3) and fit the data x,y which is in df_test
kmeans = KMeans(n_clusters=3, random_state=0).fit(df_test)

# Get the cluster centroids
kmeans.cluster_centers_

# Get the cluster labels
kmeans.labels_

# Plotting the cluster centers and the data points on a 2D plane
plt.scatter(x, y)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], c='red', marker='x')
plt.title('Data points and cluster centroids')
plt.show()

# Calculate silhouette_score
from sklearn.metrics import silhouette_score

print(silhouette_score(df_test, kmeans.labels_))

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np



# Generating the sample data from make_blobs
# This particular setting has one distinct cluster and 3 clusters placed close
# together.

'''

X, y = make_blobs(n_samples=500,
                  n_features=2,
                  centers=4,
                  cluster_std=1,
                  center_box=(-10.0, 10.0),
                  shuffle=True,
                  random_state=1)  # For reproducibility

'''

# print(X.shape)
# print(y.shape)


X = df_test
y = kmeans.labels_

range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(X['age'], X['potential'], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()